In [4]:
# from glob import glob
# import itertools
import os.path
import re
# import tarfile
# import time
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams

# from html.parser import HTMLParser
# from urllib.request import urlretrieve
# from sklearn.datasets import get_data_home
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [7]:
import spacy
import en_core_web_sm
import re
from nltk.tokenize.regexp import RegexpTokenizer

In [8]:
df = pd.read_csv('./emaildataset.csv', usecols = ['Subject','Body', 'Class'])
print(df.shape)
df.head()

(951, 3)


,Subject,Body,Class
0,Transaction no. 072558 is unresolved.,Sorry to inform that there has been only a par...,Pending
1,Order for new Cheque book,"Good morning, I want to place an order for an ...",General
2,Required money acquired. Transaction 847047 is...,Hello! This is to inform you that I have recei...,Processing
3,Asking for the details for transaction 746078,I request you to kindly send the status of my ...,Request
4,Partial payment for transaction 535918,Hello!! Greetings for the day. Status of trans...,Pending


In [9]:
true_k = df.Class.unique().shape[0]

In [10]:
true_k

6

In [11]:
nlp_ = en_core_web_sm.load()

In [12]:
nlp = spacy.load('en')

In [13]:
my_stop = {"'d", "'ll", "'m", "'re", "'s", "'ve",'a','cc','subject','http', 'gbp', 'usd', 'eur', 'inr', 'cad', 'thanks', "acc", "id", 'account', 'regards', 'hi', 'hello', 'thank you', 'greetings', 'about','above', 'across','after','afterwards','against','alone','along','already','also','although','always','am','among', 'amongst','amount','an','and','another','any','anyhow','anyone','anything','anyway','anywhere','are','around','as', 'at','back','be','became','because','become','becomes','becoming','been','before','beforehand','behind','being','below', 'beside','besides','between','beyond','both','bottom','but','by','ca','call','can','could','did', 'do', 'does', 'doing', 'done', 'down', 'due', 'during', 'each', 'eight', 'either', 'eleven', 'else', 'elsewhere', 'empty', 'enough', 'even', 'ever', 'every', 'everyone', 'everything', 'everywhere', 'except', 'few', 'fifteen', 'fifty', 'first', 'five', 'for', 'former', 'formerly', 'forty', 'four', 'from', 'front', 'full', 'further', 'get', 'give', 'go', 'had', 'has', 'have', 'he', 'hence', 'her', 'here', 'hereafter', 'hereby', 'herein', 'hereupon', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'however', 'hundred', 'i', 'if', 'in', 'indeed', 'into', 'is', 'it', 'its', 'itself', 'just', 'keep', 'last', 'latter', 'latterly', 'least', 'less', 'made', 'make', 'many', 'may', 'me', 'meanwhile', 'might', 'mine', 'more', 'moreover', 'most', 'mostly', 'move', 'much', 'must', 'my', 'myself', 'name', 'namely', 'neither', 'never', 'nevertheless', 'next', 'nine', 'no', 'nobody', 'now', 'nowhere', 'of', 'off', 'often', 'on', 'once', 'one', 'only', 'onto', 'or', 'other', 'others', 'otherwise', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 'part', 'per', 'perhaps', 'please', 'put', 'quite', 'rather', 're', 'really', 'regarding', 'same', 'say', 'see', 'seem', 'seemed', 'seeming', 'seems', 'serious', 'several', 'she', 'should', 'show', 'side', 'since', 'six', 'sixty', 'so', 'some', 'somehow', 'someone', 'something', 'sometime', 'sometimes', 'somewhere', 'still', 'such', 'take', 'ten', 'than', 'that', 'the', 'their', 'them', 'themselves', 'then', 'thence', 'there', 'thereafter', 'thereby', 'therefore', 'therein', 'thereupon', 'these', 'they', 'third', 'this', 'those', 'though', 'three', 'through', 'throughout', 'thru', 'thus', 'to', 'together', 'too', 'top', 'toward', 'towards', 'twelve', 'twenty', 'two', 'under', 'unless', 'until', 'up', 'upon', 'us', 'used', 'using', 'various', 'very', 'via', 'was', 'we', 'well', 'were', 'what', 'whatever', 'when', 'whence', 'whenever', 'where', 'whereafter', 'whereas', 'whereby', 'wherein', 'whereupon', 'wherever', 'whether', 'which', 'while', 'whither', 'who', 'whoever', 'whole', 'whom', 'whose', 'why', 'will', 'with', 'within', 'would', 'yet', 'you', 'your', 'yours', 'yourself', 'yourselves', '‘d', '‘ll', '‘m', '‘re', '‘s', '‘ve', '’d', '’ll', '’m', '’re', '’s', '’ve'}

In [14]:
def clean(text):
    
    text=text.rstrip()
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = " ".join([i for i in text.lower().split()])
    print(text)
    
    customize_stop_words = ["cc","subject","http", "gbp", "usd", "eur", "inr", "cad","thanks", "acc", "id", "account", "regards", "hi", "hello", "thank you", "greetings"]
    rem_stop = ["not", "wasn't", "hadn't", "won't", "can't", "didn't"]
    
    for w in customize_stop_words:
        nlp.vocab[w].is_stop = True
    
    for w in rem_stop:
        nlp.vocab[w].is_stop = False
    
    doc = nlp(text)
    
    normalized = " ".join(token.lemma_ for token in doc if token not in my_stop)
    
    doc = " ".join(token.orth_ for token in nlp(normalized) if not token.is_punct | token.is_space)
    return doc

In [15]:
for i in range(df.shape[0]):
    # merge subject and body strings
    df['Text_Data'] = (df['Subject'] + " " + df['Body'])

In [16]:
def converter(x):
    try:
        return ' '.join([x.lower() for x in str(x).split()])
    except AttributeError:
        return None  # or some other value

df['Text_Data'] = df['Text_Data'].apply(converter)

In [17]:
text_clean=[]

for i in range(df.shape[0]):
    text_clean.append(clean(df.loc[i]['Text_Data']))

eful for your support warm regards
partially paid the required amount for transaction i regret to inform you the i could only pay the partial amount of gbp i will definitely pay the remaining amount as soon as possible thank you for understanding regards
abrupt closure of transaction with id hey i see my transaction with id has failed i think i did everything right please look into this and reply with the reason urgently thank you and regards
want to block account i would like to know the instructions to be followed to install and setup mobile banking please reply to me with the same thanks in advance
the pending amount for transaction will reach you soon i regret to inform you the i could only pay the partial amount of eur i will definitely pay the remaining amount as soon as possible thank you for understanding regards
sorted out the transaction with id hello sincere greetings for the day i would like to inform you that my transaction has completed thank you so much for your support 

In [18]:
df['Text_Data'] = text_clean

In [19]:
df.head()

,Subject,Body,Class,Text_Data
0,Transaction no. 072558 is unresolved.,Sorry to inform that there has been only a par...,Pending,transaction no be unresolved sorry to inform t...
1,Order for new Cheque book,"Good morning, I want to place an order for an ...",General,order for new cheque book good morning i want ...
2,Required money acquired. Transaction 847047 is...,Hello! This is to inform you that I have recei...,Processing,require money acquire transaction be in proces...
3,Asking for the details for transaction 746078,I request you to kindly send the status of my ...,Request,ask for the detail for transaction i request -...
4,Partial payment for transaction 535918,Hello!! Greetings for the day. Status of trans...,Pending,partial payment for transaction hello greeting...


In [20]:
df['Text_Data'][0]

'transaction no be unresolved sorry to inform that there have be only a partial payment of amount gbp -PRON- be definitely go to receive the rest at a later point of time warm regard'

In [21]:
labels = list(df['Class'].values)

In [22]:
df = df.drop(['Subject', 'Body'], axis=1)

In [23]:
pd.DataFrame(data = df.values).to_csv("emaildataset_core.csv",index = False, header = ['Class', 'Text_Data'])

In [49]:
def stream_docs(path):
    with open(path, 'r') as csv:
        next(csv) # skip header
        for line in csv:
            data=line.split(",")
            text, label = data[1][:-1], data[0]
            yield text, label

In [50]:
next(stream_docs(path='./emaildataset_core.csv'))

('transaction no be unresolved sorry to inform that there have be only a partial payment of amount gbp -PRON- be definitely go to receive the rest at a later point of time warm regard',
 'Pending')

In [51]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    for _ in range(size):
        text, label = next(doc_stream)
        docs.append(text)
        y.append(label)
    return docs, y

In [52]:
from sklearn.feature_extraction.text import HashingVectorizer
vect = HashingVectorizer(decode_error='ignore', n_features=2**18, preprocessor=None, tokenizer=clean, alternate_sign=False)

In [53]:
from sklearn.linear_model import SGDClassifier
# SGDClassifier?
clf = SGDClassifier(max_iter=5)
per = Perceptron(verbose=10, n_jobs=-1, max_iter=5)
nb = MultinomialNB(alpha=0.01)
pa =PassiveAggressiveClassifier()

doc_stream = stream_docs(path='./emaildataset_core.csv')

In [55]:
import pyprind
pbar = pyprind.ProgBar(90)
# from sklearn import preprocessing

# classes = np.array([0, 1])
for _ in range(90):
    X_train, y_train = get_minibatch(doc_stream, size=10)
    X_train = vect.transform(X_train)
    # print(X_train.shape)
    # print(len(y_train))
    
    ## SGD
#     clf.partial_fit(X_train, y_train, classes=labels)
    
    ## Perceptron
#     per.partial_fit(X_train, y_train, classes=labels)
    
    ## MultiNB
    nb.partial_fit(X_train, y_train, classes=labels)
    
    ## Passive Aggressive Classifier
#     pa.partial_fit(X_train, y_train, classes=labels)
    
    pbar.update()

the pende amount for transaction will reach pron soon since pron transaction be still pende i want to know if there be a problem with the paperwork from pron side please let pron know at the early thank and regard
handle the transaction after payment sincere greeting i be glad to tell pron that i have finally be grant the pre approve amount through transaction pron be now be process i be grateful for pron support warm regard
upgrade to an account with more benefit hello greeting for the day so i have an account with pron branch have no and i have recently see an advertisement which state that if i become a premium customer i can avail many benefit i have decide to upgrade pron account can pron please guide pron through the step to be take in order to do this i would be very grateful if pron reply within day thank pron
process transaction have pron d acknowledge the received payment for transaction pron be finally in process i be appreciative of pron effort kind regard
issue a new chequ

ValueError: operands could not be broadcast together with shapes (951,262144) (6,262144) (951,262144) 

In [46]:
X_test, y_test = get_minibatch(doc_stream, size=51)
X_test = vect.transform(X_test)
# X_test_minmax = mm_scaler.transform(X_test)

# print('Accuracy: %.3f' % clf.score(X_test, y_test))
# print('Accuracy: %.3f' % per.score(X_test, y_test))
print('Accuracy: %.3f' % nb.score(X_test, y_test))
# print('Accuracy: %.3f' % pa.score(X_test, y_test))

payment outstanding for transaction the transaction be take too long to complete i would request pron to kindly guide pron through the further step to be take in order to complete the transaction
payment outstanding for transaction hello greeting for the day status of transaction usd for account be pende i would be grateful if pron could tell pron the cause thank a lot
implore update on transaction can pron please tell pron the amount transfer through transaction pron d thank
ask for the detail for transaction a detail update on the transaction would be highly appreciate thank and regard
partially pay the require amount for transaction there have be only a partial payment of amount usd assure pron that the rest will be pay later warm regard
urgent transaction cease i have be pron regular client and have follow the procedure for create a transaction correctly still i receive an email say that the transaction have fail i would like to know the reasoning pron transaction pron d be regard


AttributeError: 'MultinomialNB' object has no attribute 'feature_log_prob_'

In [ ]:
# print(classification_report(y_pred=clf.predict(X_test), y_true=y_test, labels=labels))
# print(classification_report(y_pred=per.predict(X_test), y_true=y_test, labels=labels))
print(classification_report(y_pred=nb.predict(X_test), y_true=y_test, labels=labels))
# print(classification_report(y_pred=pa.predict(X_test), y_true=y_test, labels=labels))

In [145]:
# use those last test samples to update our model.

In [134]:
clf = clf.partial_fit(X_test, y_test, classes=labels)
per.partial_fit(X_test, y_test, classes=labels)
nb.partial_fit(X_test, y_test, classes=labels)
pa.partial_fit(X_test, y_test, classes=labels)

### SEE HOW TO FIT!

In [ ]:
import random
clf2 = SGDClassifier(loss='log') # shuffle=True is useless here
shuffledRange = range(len(X))
n_iter = 5
for n in range(n_iter):
    random.shuffle(shuffledRange)
    shuffledX = [X[i] for i in shuffledRange]
    shuffledY = [Y[i] for i in shuffledRange]
    for batch in batches(range(len(shuffledX)), 10000):
        clf2.partial_fit(shuffledX[batch[0]:batch[-1]+1], shuffledY[batch[0]:batch[-1]+1], classes=numpy.unique(Y))


## Serialize a Python object structure

In [136]:
import joblib
import os
if not os.path.exists('./pkl_objects'):
    os.mkdir('./pkl_objects')
    
joblib.dump(vect, './pkl_objects/vectorizer.pkl')
joblib.dump(clf, './pkl_objects/clf.pkl')

['./pkl_objects/clf.pkl']

In [158]:
%%writefile tokenizer.py
import re
import spacy
import en_core_web_sm

def clean(text):    
    nlp_ = en_core_web_sm.load()
    nlp = spacy.load('en')
    my_stop = ["'d", "'ll", "'m", "'re", "'s", "'ve",'a','cc','subject','http', 'gbp', 'usd', 'eur', 'inr', 'cad', 'thanks', "acc", "id", 'account', 'regards', 'hi', 'hello', 'thank you', 'greetings', 'about','above', 'across','after','afterwards','against','alone','along','already','also','although','always','am','among', 'amongst','amount','an','and','another','any','anyhow','anyone','anything','anyway','anywhere','are','around','as', 'at','back','be','became','because','become','becomes','becoming','been','before','beforehand','behind','being','below', 'beside','besides','between','beyond','both','bottom','but','by','ca','call','can','could','did', 'do', 'does', 'doing', 'done', 'down', 'due', 'during', 'each', 'eight', 'either', 'eleven', 'else', 'elsewhere', 'empty', 'enough', 'even', 'ever', 'every', 'everyone', 'everything', 'everywhere', 'except', 'few', 'fifteen', 'fifty', 'first', 'five', 'for', 'former', 'formerly', 'forty', 'four', 'from', 'front', 'full', 'further', 'get', 'give', 'go', 'had', 'has', 'have', 'he', 'hence', 'her', 'here', 'hereafter', 'hereby', 'herein', 'hereupon', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'however', 'hundred', 'i', 'if', 'in', 'indeed', 'into', 'is', 'it', 'its', 'itself', 'just', 'keep', 'last', 'latter', 'latterly', 'least', 'less', 'made', 'make', 'many', 'may', 'me', 'meanwhile', 'might', 'mine', 'more', 'moreover', 'most', 'mostly', 'move', 'much', 'must', 'my', 'myself', 'name', 'namely', 'neither', 'never', 'nevertheless', 'next', 'nine', 'no', 'nobody', 'now', 'nowhere', 'of', 'off', 'often', 'on', 'once', 'one', 'only', 'onto', 'or', 'other', 'others', 'otherwise', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 'part', 'per', 'perhaps', 'please', 'put', 'quite', 'rather', 're', 'really', 'regarding', 'same', 'say', 'see', 'seem', 'seemed', 'seeming', 'seems', 'serious', 'several', 'she', 'should', 'show', 'side', 'since', 'six', 'sixty', 'so', 'some', 'somehow', 'someone', 'something', 'sometime', 'sometimes', 'somewhere', 'still', 'such', 'take', 'ten', 'than', 'that', 'the', 'their', 'them', 'themselves', 'then', 'thence', 'there', 'thereafter', 'thereby', 'therefore', 'therein', 'thereupon', 'these', 'they', 'third', 'this', 'those', 'though', 'three', 'through', 'throughout', 'thru', 'thus', 'to', 'together', 'too', 'top', 'toward', 'towards', 'twelve', 'twenty', 'two', 'under', 'unless', 'until', 'up', 'upon', 'us', 'used', 'using', 'various', 'very', 'via', 'was', 'we', 'well', 'were', 'what', 'whatever', 'when', 'whence', 'whenever', 'where', 'whereafter', 'whereas', 'whereby', 'wherein', 'whereupon', 'wherever', 'whether', 'which', 'while', 'whither', 'who', 'whoever', 'whole', 'whom', 'whose', 'why', 'will', 'with', 'within', 'would', 'yet', 'you', 'your', 'yours', 'yourself', 'yourselves', '‘d', '‘ll', '‘m', '‘re', '‘s', '‘ve', '’d', '’ll', '’m', '’re', '’s', '’ve']

    text=text.rstrip()
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = " ".join([i for i in text.lower().split()])
#     print(text)
    
    doc = nlp(text)
    
    normalized = " ".join(token.lemma_ for token in doc if token not in my_stop)
    
    doc = " ".join(token.orth_ for token in normalized if not token.is_punct | token.is_space)
    return doc

Overwriting tokenizer.py


In [159]:
from tokenizer import clean
joblib.dump(clean, './pkl_objects/clean.pkl')

['./pkl_objects/clean.pkl']

In [160]:
# Now, let us restart this IPython notebook and check if the we can load our serialized objects:

import joblib
tokenizer = joblib.load('./pkl_objects/clean.pkl')
vect = joblib.load('./pkl_objects/vectorizer.pkl')
clf = joblib.load('./pkl_objects/clf.pkl')
# After loading the tokenizer, HashingVectorizer, and the tranined logistic regression model, we can use it to make predictions on new data, which can be useful
# ,if we'd want to embed our classifier into a web application

In [161]:
example = ['I did not like this movie']
X = vect.transform(example)
clf.predict(X)

i did not like this movie


NameError: name 'nlp' is not defined

In [ ]:
example = ['I loved this movie']
X = vect.transform(example)
clf.predict(X)

## Implementing K-Means++

In [83]:
# parse commandline arguments
op = OptionParser()
op.add_option("--lsa",
              dest="n_components", type="int",
              help="Preprocess documents with latent semantic analysis.")
op.add_option("--no-minibatch",
              action="store_false", dest="minibatch", default=True,
              help="Use ordinary k-means algorithm (in batch mode).")
op.add_option("--no-idf",
              action="store_false", dest="use_idf", default=True,
              help="Disable Inverse Document Frequency feature weighting.")
op.add_option("--use-hashing",
              action="store_true", default=False,
              help="Use a hashing feature vectorizer")
op.add_option("--n-features", type=int, default=100000,
              help="Maximum number of features (dimensions)"
                   " to extract from text.")
op.add_option("--verbose",
              action="store_true", dest="verbose", default=False,
              help="Print progress reports inside k-means algorithm.")

print(__doc__)
op.print_help()

Automatically created module for IPython interactive environment
Usage: ipykernel_launcher.py [options]

Options:
  -h, --help            show this help message and exit
  --lsa=N_COMPONENTS    Preprocess documents with latent semantic analysis.
  --no-minibatch        Use ordinary k-means algorithm (in batch mode).
  --no-idf              Disable Inverse Document Frequency feature weighting.
  --use-hashing         Use a hashing feature vectorizer
  --n-features=N_FEATURES
                        Maximum number of features (dimensions) to extract
                        from text.
  --verbose             Print progress reports inside k-means algorithm.


In [106]:
print("Extracting features from the training dataset using a sparse vectorizer")

t0 = time()

if opts.use_hashing:
    if opts.use_idf:
        # Perform an IDF normalization on the output of HashingVectorizer
        hasher = HashingVectorizer(n_features=opts.n_features, stop_words=my_stop, alternate_sign=False, norm=None)
        vectorizer = make_pipeline(hasher, TfidfTransformer())
    
    else:
        vectorizer=HashingVectorizer(n_features=opts.n_features, stop_words=my_stop, 
                                     alternate_sign=False, norm='l2')

else:       
    vectorizer = TfidfVectorizer(max_df=0.5, max_features=opts.n_features,
                                 min_df=2, stop_words=my_stop, use_idf=opts.use_idf)
    

X = vectorizer.fit_transform(text_clean)

print("done in %fs" % (time() - t0))
print("n_samples: %d, n_features: %d" % X.shape)
print()

if opts.n_components:
    print("Performing dimensionality reduction using LSA")
    t0 = time()
    # Vectorizer results are normalized, which makes KMeans behave as
    # spherical k-means for better results. Since LSA/SVD results are 
    # not normalized, we have to redo the normalization.
    
    svd = TruncatedSVD(opts.n_components)
    normalizer = Normalizer(copy=False)
    lsa = make_pipeline(svd, normalizer)

    X = lsa.fit_transform(X)

    print("done in %fs" % (time() - t0))

    explained_variance = svd.explained_variance_ratio_.sum()
    print("Explained variance of the SVD step: {}%".format(
        int(explained_variance * 100)))

    print()

Extracting features from the training dataset using a sparse vectorizer
done in 0.041258s
n_samples: 951, n_features: 212



/home/aheli/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 'thank', 've'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [108]:
# Do the actual clustering

if opts.minibatch:
    km = MiniBatchKMeans(n_clusters=true_k, init='k-means++', n_init=1,
                         init_size=5, batch_size=5, verbose=opts.verbose)
else:
    km = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1,
                verbose=opts.verbose)

print("Clustering sparse data with %s" % km)
t0 = time()
km.fit(X)
print("done in %0.3fs" % (time() - t0))
print()

print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels, km.labels_))
print("Completeness: %0.3f" % metrics.completeness_score(labels, km.labels_))
print("V-measure: %0.3f" % metrics.v_measure_score(labels, km.labels_))
print("Adjusted Rand-Index: %.3f"
      % metrics.adjusted_rand_score(labels, km.labels_))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, km.labels_, sample_size=1000))

print()


if not opts.use_hashing:
    print("Top terms per cluster:")

    if opts.n_components:
        original_space_centroids = svd.inverse_transform(km.cluster_centers_)
        order_centroids = original_space_centroids.argsort()[:, ::-1]
    else:
        order_centroids = km.cluster_centers_.argsort()[:, ::-1]

    terms = vectorizer.get_feature_names()
    for i in range(true_k):
        print("Cluster %d:" % i, end='')
        for ind in order_centroids[i, :10]:
            print(' %s' % terms[ind], end='')
        print()

Clustering sparse data with MiniBatchKMeans(batch_size=5, compute_labels=True, init='k-means++',
        init_size=5, max_iter=100, max_no_improvement=10, n_clusters=6,
        n_init=1, random_state=None, reassignment_ratio=0.01, tol=0.0,
        verbose=False)
done in 0.139s

Homogeneity: 0.341
Completeness: 0.569
V-measure: 0.426
Adjusted Rand-Index: 0.125
Silhouette Coefficient: 0.092

Top terms per cluster:
Cluster 0: process currently transfer inform payment receive acknowledge response finally kind
Cluster 1: pay partial rest understanding assure remain later payment regret soon
Cluster 2: fail kindly detail reply complete request tell know status update
Cluster 3: finalized confirm want let acknowledge status know response email payment
Cluster 4: wrong mention error reason state new create advance wait fail
Cluster 5: priority basis detail provide urgently need update request send require


/home/aheli/anaconda3/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:1562: RuntimeWarning: init_size=5 should be larger than k=6. Setting it to 3*k
  init_size=init_size)
